### PAIR 5. NORMALIZACIÓN.


----------------

In [1]:
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt

from scipy import stats
import math 
from sklearn.preprocessing import MinMaxScaler

import warnings
warnings.filterwarnings('ignore')

pd.options.display.max_columns = None 

In [2]:
df = pd.read_pickle("data/supermarket_2.pkl")

En vuestro dataset habréis identificado unas variables predictoras y una variable respuesta. Los objetivos del pair programming de hoy son:

- Sobre la variable respuesta, en pair programmings anterior identificastéis si seguía una distribución normal o no. En caso de que no siguiera una distribución normal, normalizarla. Podéis usar el método que prefiráis o el que mejor se ajuste.

In [3]:
#Vamos a probar los métodos que conocemos, comenzando con el manual.
media = df["total"].mean()
maximo = df["total"].max() 
minimo = df["total"].min()

In [4]:
df["total_manual"] = (df["total"] - media) / (maximo - minimo )

In [5]:
stats.shapiro(df['total_manual'])

ShapiroResult(statistic=0.9087594151496887, pvalue=5.3032554860066655e-24)

Con el método manual nuestros datos no se han normalizado porque siguen teniendo un p-valor de menos de 0.05

In [6]:
#Utilizamos el método logaritmico.

df["total_log"] = df["total"].apply(lambda x: np.log(x) if x > 0 else 0)

In [7]:
stats.shapiro(df['total_log'])

ShapiroResult(statistic=0.965816855430603, pvalue=1.4293382829318885e-14)

Nuestros datos tienen un p-value de menos de 0.05, por lo que siguen sin normalizarse.

In [8]:
#Probamos con el método raiz cuadrada.

df["total_raiz2"] = df["total"].apply(lambda x: math.sqrt(x))

In [9]:
stats.shapiro(df["total_raiz2"])

ShapiroResult(statistic=0.9748885035514832, pvalue=3.937140544141293e-12)

De nuevo, nuestros datos tienen un p-value de menos de 0.05, por lo que siguen sin normalizarse.

In [10]:
#Probamos con el método SKlearn MinMaxScaler.

In [11]:
numericas = df.select_dtypes(include = np.number)
minmax = MinMaxScaler()
minmax.fit(df[['total']])

MinMaxScaler()

In [12]:
X_normalizadas = minmax.transform(df[["total"]])
X_normalizadas.min(axis=0)
X_normalizadas.max(axis=0)

array([1.])

In [13]:
numericas_minmax = pd.DataFrame(X_normalizadas, columns = ["total_minmax"])
df["total_minmax"] = numericas_minmax

In [14]:
stats.shapiro(df["total_minmax"])

ShapiroResult(statistic=0.9087594151496887, pvalue=5.3032554860066655e-24)

Con este método tampoco hemos normalizado nuestros datos.

In [15]:
#Probamos por último con el método boxcox.

df['total_boxcox'], lambda_ajustada = stats.boxcox(df['total'])

In [16]:
stats.shapiro(df['total_boxcox'])

ShapiroResult(statistic=0.9844849705696106, pvalue=8.098351322871622e-09)

Nuestros datos siguen sin ser normales con el método de Boxcox, por lo que no podremos normalizarlos.

In [17]:
#Nos quedamos solamente con el valor de total resultante de aplicar el box-cox, porque es el que nos ha dado mejor resultado.

df = df.drop(["total", "total_manual", "total_log", "total_raiz2", "total_minmax"], axis = 1)
df.head(1)

,city,customer_type,gender,product_line,unit_price,quantity,date,time,payment,rating,month,shift,total_boxcox
0,Yangon,Member,Female,Health and beauty,74.69,7,2019-01-05,13:08,Ewallet,9.1,January,Morning,17.284419


- Guardar en un csv el nuevo dataframe que habéis creado para seguir usándolo en los siguientes pair programmings.

In [18]:
df.to_pickle('data/supermarket_norm.pkl')